In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
import matplotlib as mpl

# LB1 density plot

In [ ]:
density_dat = pd.read_csv('tables_out/all_dat_tog.tsv', sep='\t').replace('EarlySomite','Early somite')
density_dat['score_lb1'] = density_dat[['score0_lb1','score1_lb1']].mean(axis=1)
density_dat['score_k9'] = density_dat[['score0_k9','score1_k9']].mean(axis=1)

mypal = {'T1-LAD':'indigo','T2-LAD':'mediumpurple','nonLAD':'lightgrey'}

f, axB = plt.subplots(figsize=(4,8))

b = sns.violinplot(x='score_lb1', y='cell_type', data=density_dat, kind='violin',
           hue='category_lb1',linewidth=0.5, inner=None, ax=axB,
                   hue_order=['T1-LAD','T2-LAD','nonLAD'],
          order=['ESCs','Mid-hindgut','Liver','Anterior primitive streak',
                'Paraxial mesoderm','Early somite','Artery progenitors',
                'Endothelial progenitors','Cardiac myocytes','Epicardium',
                'Definitive ectoderm','Border ectoderm','Midbrain progenitors'],
                  palette = mypal)
plt.legend(frameon=False, title='LAD category', bbox_to_anchor=(1, 1.15),
          ncol=1)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
axB.set(ylabel='cell type', xlabel='LB1 binding [log2(read counts)]')
plt.tight_layout()

# LB1 at LAD boundaries

Example deepTools commands:

`computeMatrix reference-point -S bigwigs/H9ESC_rep1.bigwig -R ctcf_analysis/non_to_t2_escs.bed --referencePoint center -a 500000 -b 500000 -out LB1_non_to_t2_escs.tab.gz -bs 20000


plotProfile -m LB1_non_to_t2_escs.tab.gz --outFileName LB1_non_to_t2_escs_profile.png --outFileNameData LB1_non_to_t2_escs_for_plot.tsv`

In [ ]:
def load_deeptools_dat(path, rtype, dtype):
    dat = pd.read_table(path,
                         skiprows=1, header=0, low_memory=False).replace(np.nan, 0.0).replace('--',0.0)
    dat.columns = [dtype ,rtype] + list(range(dat.shape[1]))[:-2]
    dat.columns = [dtype,rtype] + list(range(dat.shape[1]))[:-2]

    dfs = []

    for ix, row in dat.iterrows():
        dset = row[dtype]
        r_dset = row[rtype]

        dat = pd.DataFrame(row[2:].T)
        dat.columns=[f'{dtype}_sig']
        dat[dtype] = dset.split('_')[0]
        dat[rtype] = r_dset.split('_')[0]
        dat['bin'] = dat.index

        dfs.append(dat)


    tog = pd.concat(dfs, sort=False)
    tog[f'{dtype}_sig'] = tog[f'{dtype}_sig'].astype(float)
    return(tog)

In [ ]:
# deepTools has a weird quirk where it outputs additional bins

max_bin = 50

# T2 to non

non_to_t2_all = []

for ct in ['escs','parax_mesoderm','cms']:
    t2_to_non = load_deeptools_dat(f'LB1_t2_to_non_{ct}_for_plot.tsv','T2_to_non','LB1')#.query('bin > 0')
    rev_bins = t2_to_non['bin'].tolist()
    rev_bins.reverse()
    t2_to_non['bin'] = rev_bins
    non_to_t2 = load_deeptools_dat(f'LB1_non_to_t2_{ct}_for_plot.tsv','T2_to_non','LB1')#.query('bin < @max_bin')
    t2_to_non_tog = pd.concat([t2_to_non, non_to_t2], sort=False).sort_values(by='bin', ascending=False)
    t2_to_non_tog['cat'] = 'T2_to_non'
    t2_to_non_tog['ct'] = ct
    non_to_t2_all.append(t2_to_non_tog)
    
non_to_t2_all_cts = pd.concat(non_to_t2_all, sort=False)

# T2 to T1

t2_to_t1_all = []

for ct in ['escs','parax_mesoderm','cms']:
    t1_to_t2 = load_deeptools_dat(f'LB1_t1_to_t2_{ct}_for_plot.tsv','T2_to_T1','LB1')#.query('bin < @max_bin')
    rev_bins = t1_to_t2['bin'].tolist()
    rev_bins.reverse()
    t1_to_t2['bin'] = rev_bins
    t2_to_t1 = load_deeptools_dat(f'LB1_t2_to_t1_{ct}_for_plot.tsv','T2_to_T1','LB1')#.query('bin < @max_bin')
    t2_to_t1_tog = pd.concat([t2_to_t1, t1_to_t2], sort=False).sort_values(by='bin', ascending=False)
    t2_to_t1_tog['cat'] = 'T2_to_t1'
    t2_to_t1_tog['ct'] = ct
    t2_to_t1_all.append(t2_to_t1_tog)
    
t2_to_t1_all_cts = pd.concat(t2_to_t1_all, sort=False)

In [ ]:
fig = plt.figure(figsize=(8,3))
gs = mpl.gridspec.GridSpec(1, 2, width_ratios=[1, 1],
                          height_ratios=[1])

# nonLAD --> T2-LAD

axA = plt.subplot(gs[0, 0])
a = sns.lineplot(x='bin', y='LB1_sig', data=non_to_t2_all_cts.replace({'escs':'ESCs',
                                                                       'cms':'Cardiac myocytes',
                                                                       'parax_mesoderm':'Paraxial mesoderm'}), 
                 hue='ct', ci='sd', alpha=0.5, estimator='mean', 
                 hue_order=['ESCs','Paraxial mesoderm','Cardiac myocytes'],
                ax=axA)
a.set(ylabel='LB1 [log2(read counts)]', xlabel='genomic position', ylim=(-0.4, 0.7))
axA.legend(frameon=False, bbox_to_anchor = (1., -0.2))
axA.set_xticks([0, 25, 50])
axA.set_xticklabels(['-0.5Mb','boundary','+0.5Mb'])
axA.axvline(25, linestyle='--', color='grey')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
axA.annotate('',
            xy=(0.1, .98), xycoords='axes fraction',
            xytext=(0.4, .98), textcoords='axes fraction',
            arrowprops=dict(facecolor='grey', shrink=0.05, alpha=0.5),
            horizontalalignment='right', verticalalignment='top')
axA.annotate('',
            xy=(0.9, .98), xycoords='axes fraction',
            xytext=(0.6, .98), textcoords='axes fraction',
            arrowprops=dict(facecolor='grey', shrink=0.05, alpha=0.5),
            horizontalalignment='right', verticalalignment='top')

# T2-LAD --> T1-LAD

axB = plt.subplot(gs[0, 1])
b = sns.lineplot(x='bin', y='LB1_sig', data=t2_to_t1_all_cts.replace({'escs':'ESCs',
                                                                       'cms':'Cardiac myocytes',
                                                                       'parax_mesoderm':'Paraxial mesoderm'}), 
                 hue='ct', ci='sd', alpha=0.5, estimator='mean', 
                 hue_order=['ESCs','Paraxial mesoderm','Cardiac myocytes'],
                ax=axB)
b.set(ylabel='LB1 [log2(read counts)]', xlabel='genomic position', ylim=(-0.4, 0.7))
axB.legend(frameon=False, bbox_to_anchor = (1., -0.2))
axB.set_xticks([0, 25, 50])
axB.set_xticklabels(['-0.5Mb','boundary','+0.5Mb'])
axB.axvline(25, linestyle='--', color='grey')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
axB.annotate('',
            xy=(0.1, .98), xycoords='axes fraction',
            xytext=(0.4, .98), textcoords='axes fraction',
            arrowprops=dict(facecolor='grey', shrink=0.05, alpha=0.5),
            horizontalalignment='right', verticalalignment='top')
axB.annotate('',
            xy=(0.9, .98), xycoords='axes fraction',
            xytext=(0.6, .98), textcoords='axes fraction',
            arrowprops=dict(facecolor='grey', shrink=0.05, alpha=0.5),
            horizontalalignment='right', verticalalignment='top')
plt.tight_layout()